In [ ]:
import pandas as pd
df=pd.read_csv("/kaggle/input/costa-rican-household-poverty-prediction/train.csv")
df_test=pd.read_csv("/kaggle/input/costa-rican-household-poverty-prediction/test.csv")

### Data Prepration

In [ ]:
df.head()

## Identify the output variable.

In [ ]:
#checking the size
print(df.shape)
print(df_test.shape)

Train has 143 column where as test has 142 column ,which means train has output column on it.

In [ ]:
#checking the output column my comparing the column names from both train and test
import numpy as np
np.setdiff1d(df.columns,df_test.columns)

## 1> Handling the missing value

In [ ]:
#checking the missing value
def miss_val(df):
 # Miss_val = [col for col in df_test.columns if df[col].isnull().any() == True]
#print(Miss_val)
   miss_col=df[[col for col in df.columns if df[col].isnull().any() == True]].isna().sum()
#miss_col=df[[col for col in df.columns if df[col].isnull().any() == True]].isna().sum()
   return miss_col

print(miss_val(df))
print('*************')
print(miss_val(df_test))

### Handling the above 5 missing column:
out of those 5 ,'v2a1', 'v18q1', 'rez_esc',  has  almost 70% missing data .We will check what kind of data these fiels stores and if there is any way to fill all these null value.

rez_esc - Years behind in school

v18q1 - number of tablets household owns

v2a1 - Monthly rent payment

meaneduc:average years of education for adults


SQBmeaned:square of the mean years of education of adults (>=18) in the household



In [ ]:
#Handling missing value in rez_esc
def miss_val_rez(df):
#df.rez_esc.value_counts()
 df.loc[((df['age'] >= 18) | (df['age'] < 5)) & (df['rez_esc'].isnull()), 'rez_esc'] = 0
 df.loc[df['rez_esc'] > 5, 'rez_esc'] = 5
 df['rez_esc'] = df['rez_esc'].fillna(0)
 return df.rez_esc.isna().sum()

print(miss_val_rez(df))
print(miss_val_rez(df_test))

In [ ]:
#Handling missing value in v18q1
df['v18q1'] = df['v18q1'].fillna(0)
df_test['v18q1'] = df_test['v18q1'].fillna(0)

In [ ]:
#Handling missing value in v2a1
# Fill in households that own the house with 0 rent payment

def miss_val_v2a1(df):
  df.loc[(df['tipovivi1'] == 1), 'v2a1'] = 0
  df['v2a1-miss'] = df['v2a1'].isnull()
  df['v2a1'] = df['v2a1'].fillna(0)
  return df.v2a1.isna().sum()
print(miss_val_v2a1(df))
print(miss_val_v2a1(df_test))

In [ ]:
df[['meaneduc', 'SQBmeaned']].describe()

Both the column represent education ,so using simple imputer will work for us and also there is not much of a deviation in the data
'SQBmeaned' has outlier values ,so we will use simple imputer with median .Again there are multiple ways to do this but here we are using simpleimputer.

In [ ]:
#handling missing values for column 'meaneduc', 'SQBmeaned' using iterative imputer


from sklearn.impute import SimpleImputer
import numpy as np

def miss_val_others(df):
  df1=df[['meaneduc', 'SQBmeaned']]
  Imp = SimpleImputer(missing_values=np.NaN, strategy='median')
  df2=pd.DataFrame(Imp.fit_transform(df1))
  df2.columns=['meaneduc', 'SQBmeaned']
#assigning values to the original dataframe
  df.meaneduc=df2.meaneduc
  df.SQBmeaned=df2.SQBmeaned
  
  return df[['meaneduc', 'SQBmeaned']].isna().sum()
print(miss_val_others(df))
print(miss_val_others(df_test))

In [ ]:
#Final check of missing value  in both df and df_test data
print(df[[col for col in df.columns if df[col].isnull().any() == True]].isna().sum())
print(df_test[[col for col in df_test.columns if df_test[col].isnull().any() == True]].isna().sum())

## 2> Understanding the type of data for Model building 

In [ ]:
# checking the datatypes of all the  features in the dataset
print(df_test.select_dtypes('object').shape)
print('********')
print(df_test.select_dtypes('int64').shape)
print('********')
print(df.select_dtypes('float').shape)
print('********')
print(df.select_dtypes('bool').shape)


Analysis:There are 130 integer columns, 8 float columns, and 5 object columns.

The object columns  cannot be fed directly into a machine learning model.So we neet to change the string value using label encoding or hot encoding.
 


In [ ]:
# checking for categorical features in the dataset
df_test.select_dtypes('object').head()


Analysis: Id and Idhogar are indentifiers.Id is for all the individual and idhogar is for all the individual houehold .Later we will drop these column  while defining x and y.

After analysing column  we found the other 3 columns (dependency,edjefe,edjefa) has a bunch of string and numbers .we will encode  no and yes to 0 and 1.We wn't worry about ID and Idhogar as they are just identifier columns



In [ ]:
#replacing yes and no with 0 and 1 in the dataset
def encod_rep(df):
  df[['dependency','edjefe','edjefa']] = df[['dependency','edjefe','edjefa']].replace(['yes','no'],['1','0']).astype(np.float)
  return df.select_dtypes('object').head()
print(encod_rep(df))
print('*********')
print(encod_rep(df_test))

## 3>Check if there are any biases in your dataset.

In [ ]:
import seaborn as sns
sns.countplot(x="Target", data=df)

The data is divided into the below 4 categories.

1 : extreme  poor

2:  very poor

3:  poor

4:  middle class


The majority of our data comed under Traget=4 ,which makes this dataset as an imbalance data.We will use Random forest to handle the imbalance data.



#### Performance of the standard algorithm is not great on imbalanced classification problems,so we will use Random forest classifier for our data model.

## 4> Check if there is a house without a family head.

In [ ]:
#checking the idhoghar without a famiily head.Idhogar is a family identifier.
def filter_nohead(df):
  A=df.groupby(['idhogar'])['parentesco1'].sum().to_frame('counta').reset_index()
  no_head=A[A.counta==0]
  no_head_l=list(no_head["idhogar"])
  df=df[~df["idhogar"].isin(no_head_l)]
  return df.shape
print(filter_nohead(df))
print(filter_nohead(df_test))


we have 23 household in df and 26 records in df_test  without a family head.We won't need these records for our model training and hence we have reomved it from our datset

## 5> Check whether all members of the house have the same poverty level(Target ).

In [ ]:
#df.idhogar.value_counts().sort_values(ascending=False)
#Check whether all members of the house have the same poverty level or not 
df_check=df[['idhogar','parentesco1','Target']]
df_check_unique=df_check[['idhogar','Target']].drop_duplicates().groupby(['idhogar']).agg(['mean', 'count']).reset_index()
df_check_unique.columns=['idhogar','Targ_Mean','Tar_Count']
df_check_unique=df_check_unique[df_check_unique.Tar_Count>1]#if the mean 
df_check_unique.reset_index(drop=True, inplace=True)
df_check_unique 


idhogar column is a unique Household level identifier.We need to make sure that each idhogar has same 'Target' value.

#The logic : dropping all the duplicate row in df_check[['idhogar','Target']] will leave us with the idhogar where there are  more than one type of Target value.The count greater than 1 will give us all the idhogar where we have 2 Target values.

 
We can see that there are 85 idhoghar  where  the Target values are diffrent.To handle this ,we will assign the Target value of parentesco1,household head  of the same idhoghar 
 


## 6> Set poverty level of the members and the head of the house within a family.

In [ ]:
#getting the target  for head parent for all those 85 idhoghar and creating a new dataframe
df_filter=df.idhogar.isin(df_check_unique.idhogar)
df_filter1=df[['idhogar','parentesco1','Target']][df_filter & df.parentesco1==1]
df_filter1.reset_index(drop=True, inplace=True)
df_filter1



In [ ]:
#replacing df.Target  based on df_filter1 values
df1 = df.set_index('idhogar')
df2 = df_filter1.set_index('idhogar')
df1.update(df2)
df1.reset_index(inplace=True)


In [ ]:
#checking the idhogar before and after logic:
print(df.Target[df.idhogar=='6833ac5dc'])
print(df1.Target[df1.idhogar=='6833ac5dc'])

In [ ]:
#getting the x and y from df

x=df1.drop(columns=['Id','idhogar','Target'])
y=df1.Target

In [ ]:
#getting th x from df_test for predicyion
x_t=df_test.drop(columns=['Id','idhogar'])
x_t.shape

In [ ]:

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

train_X,test_x,train_Y,test_y=train_test_split(x,y,random_state=0)

In [ ]:
#standerizing the data
stdSC=StandardScaler()
stdSC.fit(train_X)
train_new_X=pd.DataFrame(stdSC.transform(train_X))
test_x_std=stdSC.transform(test_x)

In [ ]:
from sklearn.ensemble import RandomForestClassifier

rfModel=RandomForestClassifier(n_jobs=-1,n_estimators=200,random_state=7)
rfModel.fit(train_X,train_Y)
rfModel.score(train_X,train_Y)
rfModel.score(test_x,test_y)
#rfModel.predict(x_t)

In [ ]:
#Crosschecvking the accuracy of the model for both train and test
pd.crosstab(test_y,rfModel.predict(test_x))

In [ ]:
#predicting the Target for test datset df_test
pd.DataFrame(rfModel.predict(x_t)).value_counts()

In [ ]:
A=pd.DataFrame({'ID':df_test.Id,'Target':rfModel.predict(x_t).astype(np.int64)})

In [ ]:
A.to_csv("submission.csv",index=False)

In [ ]:
sns.countplot(x="Target", data=A).set_title('Target for test')